In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.definitions import ROOT_DIR
from src.data.download import download_from_groningen
from src.data.utils import head, line_count
from src.data.quality import check_grid_spacing
from src.visualization.visualize import plot_cartesian_gridded_surface

# The Groningen field

In this section, we present some of the relevant and interesting facts of the Groningen gas field, extracted from the field's geologic overview by de Jager and Visser (2017). The Groningen field data is vast, but we can use this information to find the files needed for our study. 

The Groningen gas field was discovered in 1959 by the Slochteren-1 well. This well primary target was to test the gas trend in the Basal Zechstein carbonates, which resulted in tight basinal facies. However, the underlying Rotliegend sandstones unexpectedly found a large volume of gas. The initial recoverable gas reserves are estimated at 2900 bcm (~100 TCF). Additional field information is summarized in table 1.

Table 1: Groningen field summary.

|               | Description |
| ------------- | ----------- |
| **Structure** | Faulted anticline (4-way). The closure formed during the late Triassic or Jurassic. The primary faults have normal displacement and trend NNW–SSE. Secondary fault trends run E-W and N-S. |
| **Seal** | The top seal consists of carbonates and anhydrites layers from the Zechstein Formation. Laterally, the gas accumulation is mainly constrained by faults. |
| **Source** | Main source rocks (Carboniferous age) are the underlying and juxtaposed Westphalian Coal Measures and the basal Namurian organic shales. The primary expulsion event occurred during the Late Jurassic - Early Cretaceous. |
| **Reservoir** | Upper Permian Rotliegend reworked aeolian and fluvial sediments. High degree of sorting. Porosity ranges from 10-24 %, and permiability from 1-1000 mD. Three facies: Sandstones, conglomerates, and mudstones. |
| **FWL** | Changes by fault block, from 2972-3016 m TVNAP (NAM, 2016).|


# The Groningen data

In 2020, Nederlandse Aardolie Maatschappij (NAM) released the Groningen gas field geological model throught the Utrecht University data publication platform under a CC BY 4.0 license. These data consists of well logs, seismic horizons, a PSDM seismic cube, and the field's geocellular model, all embedded in a Petrel project. In the same year, Data Underground (2020) forked NAM's project, and made most of its data accessible in common sharing formats (CSV, LAS, SEGY, etc.). In this work, we access the data in the Data Underground repository.

The first step in our workflow is to download the data that we will use to recreate the seismic uncertainty analysis (SUA) surfaces. The typical result of a SUA is a set of P10-P50-P90 values that describe the in-place rock volume distribution of a reservoir, based on multiple iterations of the target's structural top which are derived from equiprobable imaging velocity models.

In our case, we are interested in creating multiple iterations of the reservoir structural top, so we should be looking for horizons and well tops that correspond to the Rotliegend, in addition to any other information (images, reports, README files) that will help us understand the data. 


# Data download

The data files are hosted on an Amazon's S3 bucket and can be downloaded by requesting a URL with the base repository location plus a given file name. The base repository location is

`https://swung-hosted.s3.ca-central-1.amazonaws.com/`

By reading the [Data Underground](https://dataunderground.org/dataset/groningen-open-fork) fork we find that there is a file named `groningen/README.txt`. Let's download and read this file to inform our selection of files to download.

For convenience, we added the function `download_from_groningen`, that does the URL building and requesting for us. Let's use it to dowload the README file.

In [ ]:
files_to_download = ["groningen/README.txt"]

# Local place to save the downloaded files
dst_dir = ROOT_DIR / "data/external/"

In [ ]:
head(dst_dir / "groningen/README.txt", max_line_count=30)

Excellent! Based on the README file, we know that the target sesimic horizon (Top Rotliegend) name is `RO_T`. To complement this information, let's download the file `groningen/FILENAMES.txt`, that lists the names of the files stored in the data repository. We can use this file name list to select the files to download.


In [ ]:
# List with the file names to download
files_to_download = ["groningen/FILENAMES.txt"]

In [ ]:
download_from_groningen(files_to_download, dst_dir, overwrite=False)

If all ran without errors, there should be a new file under `<project_dir>/groningen/FILENAMES.txt`.

We can open this text file to explore all of its content, looking for data useful to our study, that is, the top reservoir surface, well tops, and any pictures.

In [ ]:
files_to_download = [
    "groningen/README.txt",
    "groningen/FILENAMES.txt",
    "groningen/3DGrid/3D_Grid_Export_settings.PNG",
    "groningen/3DGrid/3D_Grid_Horizon_order.png",
    "groningen/Formation_tops/Groningen__Formation_tops__EPSG_28992.csv",
    "groningen/Horizon_Interpretation/DCAT201605_R3136_CK_B_pk_depth",
    "groningen/Horizon_Interpretation/DCAT201605_R3136_NS_B_tr_depth",
    "groningen/Horizon_Interpretation/DCAT201605_R3136_RNRO1_T_pk_depth",
    "groningen/Horizon_Interpretation/DCAT201605_R3136_RNRO1_T_pk_t",
    "groningen/Horizon_Interpretation/DCAT201605_R3136_ZE_T_na_depth",
    "groningen/Horizon_Interpretation/RO____T",
    "groningen/Seismic_Volume/R3136_15UnrPrDMkD_Full_D_Rzn_RMO_Shp_vG.SEGY",
]

In [ ]:
download_from_groningen(files_to_download, dst_dir, overwrite=False)

# Data exploration

Let's first look at the downloaded pictures:

Figure 1: Static model horizon order (NAM, 2020).

<img src="figures/3D_Grid_Horizon_order.png"  alt="Static model horizon order" width="30%" height="30%" title="Static model horizon order">


Figure 2: 3D grid export settings (NAM, 2020).

<img src="figures/3D_Grid_Export_settings.PNG"  alt="3D grid export settings" width="30%" height="30%" title="3D grid export settings">

From figure 1 and 2, we learn that the top of the reservoir model corresponds to the Top Rotliegend horizon, and that the simulation grid is not rotated, so possibly the horizon grids are also not rotated.

## Top Rotliegend

Before we start working with the Top Rotliegend horizon, we need to ensure that we can load it correctly. For this purpose, we will use the Top Rotliegend map in Kortekaas and Jaarsma (2017) as the reference (figure 3). 

Figure 3: Top Rotliegend reference map (Kortekaas and Jaarsma, 2017).

<img src="figures/Top_Rotliegend_2017_Kortekaas_Jaarsma.jpeg"  alt="Reference Top Rotliegend" width="50%" height="50%" title="Reference Top Rotliegend">

What a pretty map! Now our goal is to recreate this map from the downloaded horizon. First, lets look at the file `RO____T` file.

In [ ]:
interp_RO_T_path = dst_dir / "groningen/Horizon_Interpretation/RO____T"

line_count(interp_RO_T_path)

That is a lot of lines! Let's check the first few lines.

In [ ]:
head(interp_RO_T_path)

OK, there is no header, but the data is tabular, space separated, and the columns seem to be: Inline, Crossline, Easting, Northing, and Depth. Let's use this observations to load the data into a Pandas dataframe.

In [ ]:
col_names = ["inline", "xline", "easting", "northing", "depth"]
ro_t = pd.read_csv(interp_RO_T_path, sep=r"\s+", header=None, names=col_names)

In [ ]:
ro_t.info()

Typically, seismic horizons are created and exported in a regular grid, i.e. the seismic grid. Let's check the grid dimensions.

In [ ]:
unique_northing = check_grid_spacing(ro_t.northing)

What!? So most grid northing lenght is 50 m, but there are a few grid rows with 75 m length. Weird. Let's repeat the exercise for the easting direction.

In [ ]:
unique_easting = check_grid_spacing(ro_t.easting)

Again, most grid easting columns size are 50 m, but a few have 75 m in lenght. Let's see where these odd sized grid rows and columns post in a map.

In [ ]:
unique_northing_diff_mask = np.diff(unique_northing) != 50

unique_northing_off_values =  unique_northing[:-1][unique_northing_diff_mask]

In [ ]:
unique_easting_diff_mask = np.diff(unique_easting) != 50
unique_easting_off_values =  unique_easting[:-1][unique_easting_diff_mask]

In [ ]:
vmin = ro_t.northing.min()
vmax = ro_t.northing.max()
xmin = ro_t.easting.min()
xmax = ro_t.easting.max()

In [ ]:
title = "Top Rotliegend"
fig, ax = plt.subplots(figsize=(14, 14))
im = plot_cartesian_gridded_surface(df=ro_t, ax=ax, title=title)
ax.hlines(unique_northing_off_values,xmin=xmin, xmax=xmax)
ax.vlines(unique_easting_off_values, ymin=vmin, ymax=vmax)
fig.colorbar(im, ax=ax, label="Depth (m)")
plt.show()

Well, the good news is that we can create a map similar to the reference Top Rotliegend presented in Kortekaas and Jaarsma (2017).

In [ ]:
title = "Top Rotliegend"
fig, ax = plt.subplots(figsize=(14, 14))
im = plot_cartesian_gridded_surface(df=ro_t, ax=ax, title=title)
fig.colorbar(im, ax=ax, label="Depth (m)")
plt.savefig("figures/Loaded_Top_RO.png")




<img src="figures/Top_Rotliegend_2017_Kortekaas_Jaarsma.jpeg"  alt="Reference Top Rotliegend" width="35%" height="35%" title="Reference Top Rotliegend">

<img src="figures/Loaded_Top_RO.png"  alt="Loaded Top Rotliegend" width="50%" height="50%" title="Loaded Top Rotliegend">